In [4]:
import requests
import os
from importlib.machinery import SourceFileLoader
import pandas as pd
import logging
import torch


try:
    config = SourceFileLoader("config", "config.py").load_module()
    os.environ['OPENAI_API_KEY'] = config.OPENAI_API_KEY

    bungie_api_key = config.BUNGIE_API_KEY
    endpoint = "https://www.bungie.net/Platform/Content/Rss/NewsArticles/{pageToken}/"
    page_token = "0"
    include_body = True
    headers = {
        "X-API-Key": bungie_api_key
    }
    params = {
        "includebody": include_body
    }

    results = []

    while page_token is not None:
        response = requests.get(endpoint.format(pageToken=page_token), headers=headers, params=params)

        if response.status_code == 200:
            json_response = response.json()
            if 'NewsArticles' in json_response['Response']:
                results.extend(json_response['Response']['NewsArticles'])
            else:
                print("No NewsArticles found in the response.")
                break
            page_token = json_response['Response']['NextPaginationToken']
        else:
            print("Error:", response.status_code)
            print("Response:", response.text)
            break
except Exception as e:
    print("An error occurred:", str(e))


An error occurred: 'NextPaginationToken'


In [5]:
results[0]

{'Title': 'Special-Edition Community Focus: Nine',
 'Link': '/7/en/News/Article/community-focus-nine-rivens',
 'PubDate': '2023-07-14T00:00:00Z',
 'UniqueIdentifier': 'bltc109a5d14991a686',
 'Description': 'From Witherhoard to Withermore; meet Nine. ',
 'HtmlContent': '<p>Just as we promised in last week’s TWID, we’ve got Bianca Ojeda here, better known as Nine to talk a little bit more about her incredible art and the amazing work she’s doing with us<a href="https://www.bungie.net/7/en/News/article/7_13_23_twid" target="_blank"> for the community voted Witherhoard ornament</a>. Here’s where you’ll learn more about the artist behind the ornament and her journey throughout the world of Destiny.</p><p>From Dragon Ball Z to Destiny 2, from aspiring artist to working alongside the studio behind some of her favorite games—meet the creator behind the art: Nine!</p><h4><strong>To start off, tell us a little bit about yourself! Who are you? What do you like to do? How did you get your start in

In [6]:
# Convert the results list into a DataFrame
df = pd.DataFrame(results)


In [7]:
import re
pattern = r'Update|Hotfix'
df = df[df['Title'].str.contains(pattern, regex=True, flags=re.IGNORECASE)]
df = df.iloc[0:25].reset_index()
# Print the filtered DataFrame
df['Title']

0                         Destiny 2 Update 7.1.0.3
1                         Destiny 2 Update 7.1.0.2
2                         Destiny 2 Hotfix 6/02/23
3                         Destiny 2 Update 7.1.0.1
4                         Destiny 2 Hotfix 5/26/23
5      Destiny 2 Update 7.1.0 - Season of the Deep
6                         Destiny 2 Hotfix 7.0.5.3
7                         Destiny 2 Hotfix 7.0.5.2
8                         Destiny 2 Hotfix 7.0.5.1
9                         Destiny 2 Update 7.0.5.0
10                        Destiny 2 Hotfix 7.0.0.7
11                        Destiny 2 Hotfix 7.0.0.6
12                       Lightfall Crucible Update
13                        Destiny 2 Hotfix 7.0.0.5
14                        Destiny 2 Hotfix 7.0.0.3
15                        Destiny 2 Update 7.0.0.1
16                        Destiny 2 Hotfix 6.3.0.7
17    Economy Updates And More Coming In Lightfall
18                        Destiny 2 Hotfix 6.3.0.5
19                        Desti

In [8]:
df["HtmlContent"].iloc[0]

"<h2>Activities</h2><h4>CRUCIBLE</h4><ul><li>Fixed an issue where the Powerlifter Triumph could be completed before gilding the Glorious Seal.</li><li><strong>Competitive</strong>: Improved matchmaking times at higher skill tiers.</li><li><strong>Trials of Osiris</strong>: Fixed an issue where some players were unable to access the game mode while on step 3 of the introductory quest.</li><li><strong>Trials of Osiris</strong>: Increased the drop rate of Adept weapons on 7-win cards after going Flawless that week.</li></ul><h4>RAIDS AND DUNGEONS</h4><ul><li><strong>Ghosts of the Deep</strong>: Fixed an issue where players could respawn outside the final encounter's arena.</li></ul><h4>COMMENDATIONS</h4><ul><li>Fixed an issue where players could not access the Commendations screen after using the Wish Wall in The Last Wish Raid.</li><li>Fixed an issue where players could not access the Commendations screen after finishing a Deep Dive activity.</li></ul><h4>FISHING</h4><ul><li>Fixed an iss

In [9]:
from bs4 import BeautifulSoup


def extract_list_from_html(html):
    soup = BeautifulSoup(html, 'html.parser')
    list_items = soup.find_all('li')
    result = [li.get_text(strip=True) for li in list_items]
    return result

def clean_html(text):
    cleantext = re.sub(r'([.,?!;:])((?!\s)|$)', r'\1 ', text)
    return cleantext

def parse_html_content(df):
    df['parsed_content'] = df['HtmlContent'].apply(extract_list_from_html)
    df['joined_content'] = df['parsed_content'].apply(lambda x: ' '.join(x))
    df['joined_content'] = df['joined_content'].apply(clean_html)
    return df

In [10]:
# Parse HTML content and create new column
df = parse_html_content(df)

In [11]:
df = df[['Title', 'Link', 'PubDate', 'UniqueIdentifier', 'Description',
       'joined_content']]

In [12]:
print(df.iloc[0]["joined_content"])

Fixed an issue where the Powerlifter Triumph could be completed before gilding the Glorious Seal. Competitive: Improved matchmaking times at higher skill tiers. Trials of Osiris: Fixed an issue where some players were unable to access the game mode while on step 3 of the introductory quest. Trials of Osiris: Increased the drop rate of Adept weapons on 7-win cards after going Flawless that week. Ghosts of the Deep: Fixed an issue where players could respawn outside the final encounter's arena. Fixed an issue where players could not access the Commendations screen after using the Wish Wall in The Last Wish Raid. Fixed an issue where players could not access the Commendations screen after finishing a Deep Dive activity. Fixed an issue where the new Exotic fish Whispering Mothcarp could not be obtained. Updated the Cenotaph Mask description to correctly state its perk behavior. The description has been updated to the following: Steadily reloads a portion of your equipped Trace Rifle's maga

In [13]:

import tiktoken  

tokenizer = tiktoken.get_encoding('p50k_base')

# create the length function
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=50,
    length_function=tiktoken_len,
    separators=["\n\n", "\n", " ", ""]
)


In [14]:
import os
from importlib.machinery import SourceFileLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
config = SourceFileLoader("config", "config.py").load_module()
os.environ['OPENAI_API_KEY'] = config.OPENAI_API_KEY

In [15]:
pinecone_api_key = config.pinecone_api_key

In [16]:
import pinecone
# connect to pinecone environment
pinecone.init(
    api_key = pinecone_api_key,
    environment = "us-west1-gcp-free"
)


c:\Users\jorda\AppData\Local\Programs\Python\Python311\Lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [17]:
index_name = "extractive-question-answering"

In [18]:
pinecone.list_indexes()

['extractive-question-answering']

In [19]:
#pinecone.delete_index("extractive-question-answering")

In [26]:
# check if the extractive-question-answering index exists
if index_name not in pinecone.list_indexes():
    # create the index if it does not exist
    pinecone.create_index(
        index_name,
        dimension=1536,
        metric="cosine"
    )

# connect to extractive-question-answering index we created
index = pinecone.Index(index_name)

In [21]:
from langchain.embeddings.openai import OpenAIEmbeddings

model_name = 'text-embedding-ada-002'

embed = OpenAIEmbeddings(
    openai_api_key=os.environ['OPENAI_API_KEY']
)


In [24]:
docs = []
for index, row in df.iterrows():
    doc = {
        'content': row['joined_content'],  
            'name': row['Title'],
            'link': row['Link'],
            'pub_date': row['PubDate'],
            'unique_id': row['UniqueIdentifier'],
           'description': row['Description']
        }
    docs.append(doc)
 

In [27]:
from tqdm.auto import tqdm
from uuid import uuid4

batch_limit = 100

texts = []
metadatas = []

for i, record in enumerate(tqdm(docs)):
    # first get metadata fields for this record
    metadata = {
            'name': record['name'],
            'source': record['link'],
            'pub_date': record['pub_date'],
            'unique_id': record['unique_id'],
           'description': record['description']
        }
    # now we create chunks from the record text
    record_texts = text_splitter.split_text(record['content'])
    # create individual metadata dicts for each chunk
    record_metadatas = [{
        "chunk": j, "text": text, **metadata
    } for j, text in enumerate(record_texts)]
    # append these to current batches
    texts.extend(record_texts)
    metadatas.extend(record_metadatas)
    # if we have reached the batch_limit we can add texts
    if len(texts) >= batch_limit:
        ids = [str(uuid4()) for _ in range(len(texts))]
        embeds = embed.embed_documents(texts)
        index.upsert(vectors=zip(ids, embeds, metadatas))
        texts = []
        metadatas = []


  0%|          | 0/25 [00:00<?, ?it/s]

In [28]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 1382}},
 'total_vector_count': 1382}

In [29]:
from langchain.vectorstores import Pinecone

text_field = "text"

# switch back to normal index for langchain
index = pinecone.Index(index_name)

vectorstore = Pinecone(
    index, embed.embed_query, text_field
)


In [30]:
query = "what were the changes to the immortal?"

vectorstore.similarity_search(
    query,  # our search query
    k=3  # return 3 most relevant docs
)

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).


[Document(page_content="(crit damage goes from 21. 8 to 21). The Immortal Reduced base range value by 10. Bows Fixed an issue where the Arsenic Bite Bow would display an incorrect charge time on its tooltip. Updated the stats on the Tyranny of Heaven Bow to better compete with the current selection of Lightweight Bows. Miscellaneous and Visuals Fixed an issue where the glow when firing on the Nessa's Oblation Legendary Shotgun was not as bright as intended. Fixed an issue where the Nasreddin and", metadata={'chunk': 80.0, 'description': 'The one about Season of the Deep.', 'name': 'Destiny 2 Update 7.1.0 - Season of the Deep', 'pub_date': datetime.datetime(2023, 5, 23, 16, 30, tzinfo=tzutc()), 'source': '/7/en/News/Article/season-deep-update-7-1-0', 'unique_id': 'blt56002d0e3c869908'}),
 Document(page_content="(crit damage goes from 21. 8 to 21). The Immortal Reduced base range value by 10. Bows Fixed an issue where the Arsenic Bite Bow would display an incorrect charge time on its too

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

# completion llm
llm = ChatOpenAI(
    openai_api_key=config.OPENAI_API_KEY,
    model_name='gpt-3.5-turbo',
    temperature=0.0
)

from langchain.chains import RetrievalQAWithSourcesChain

qa_with_sources = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)


NameError: name 'config' is not defined

In [ ]:
query = "Answer the following question with as much detail as possible based only on the information provided. list all the changes to the immortal. dont include any information not related to the query"
qa_with_sources(query)

NameError: name 'qa_with_sources' is not defined

In [ ]:
from langchain.llms import OpenAI
from langchain import LLMChain

davinci = OpenAI(model_name='text-davinci-003')

from langchain import PromptTemplate
from langchain import FewShotPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector

# create our examples
examples = [
    {
        "query": "What changes were made to the Dazzling Iridescence Trials emblem?",
        "answer": "The Dazzling Iridescence Trials emblem had an issue where it could be awarded from the Flawless chest without completing the necessary requirements. This issue has been fixed in the latest patch. Players who have already acquired the emblem will need to complete the requirements after this patch goes live in order to equip the emblem."
    }, 
    {
        "query": "What is the new change regarding Gyrfalcon's Hauberk's reserve overshield?",
        "answer": "Gyrfalcon's Hauberk's reserve overshield now deploys when a player uses Ensnaring Slam. This is a change introduced in the latest patch."
    },
    {
        "query": "What were the issues fixed regarding Arbor Warden?",
        "answer": "Two issues related to Arbor Warden were fixed in this patch. One issue caused the player to become frozen which would disable Arbor Warden's Exotic perk. The other issue was causing the Juggernaut aspect to not interact with your class ability energy correctly. Both of these issues have now been addressed."
    },
    {
        "query": "What changes were made for PvP in the latest patch?",
        "answer": "In the latest patch, several PvP-related issues were fixed. For example, Verity's Brow, which was not functioning in PvP, has been fixed. An issue where Athrys's Embrace's buff icon wasn't appearing in the HUD in PvP has also been fixed. The Collective Action weapon perk's damage against Guardians has been reduced to 10% from 20%. These are some of the changes made for PvP in this patch."
    },
    {
        "query": "Were there any changes related to 'Iron Banner Special/Power weapons'?",
        "answer": "Yes, an issue where some older Iron Banner Special/Power weapons had lost their scope glint has been fixed in this latest patch."
    },
]

# create a example template
example_template = """
User: {query}
AI: {answer}
"""

# create a prompt example from above template
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)


# now break our previous prompt into a prefix and suffix
# the prefix is our instructions
prefix = "You are a knowledgeable game assistant trained to answer queries about the latest patch notes. "
# and the suffix our user input and output indicator
suffix = """
User: {query}
AI: """

query = "What were the changes to the immortal"


example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=200  # increased max length
)

# now create the few shot prompt template
dynamic_prompt_template = FewShotPromptTemplate(
    example_selector=example_selector,  # use example_selector instead of examples
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n"
)

print(dynamic_prompt_template.format(query=query))



You are a knowledgeable game assistant trained to answer queries about the latest patch notes. You have been trained on the following patch notes:

User: What changes were made to the Dazzling Iridescence Trials emblem?
AI: The Dazzling Iridescence Trials emblem had an issue where it could be awarded from the Flawless chest without completing the necessary requirements. This issue has been fixed in the latest patch. Players who have already acquired the emblem will need to complete the requirements after this patch goes live in order to equip the emblem.


User: What were the changes to the immortal
AI: 


In [ ]:
query = "What were the changes to the immortal"
qa_with_sources(dynamic_prompt_template.format(query=query))

{'question': 'You are a knowledgeable game assistant trained to answer queries about the latest patch notes. You have been trained on the following patch notes:\n\nUser: What changes were made to the Dazzling Iridescence Trials emblem?\nAI: The Dazzling Iridescence Trials emblem had an issue where it could be awarded from the Flawless chest without completing the necessary requirements. This issue has been fixed in the latest patch. Players who have already acquired the emblem will need to complete the requirements after this patch goes live in order to equip the emblem.\n\n\nUser: What were the changes to the immortal\nAI: ',
 'answer': 'The changes made to the Dazzling Iridescence Trials emblem were that it had an issue where it could be awarded from the Flawless chest without completing the necessary requirements. This issue has been fixed in the latest patch. Players who have already acquired the emblem will need to complete the requirements after this patch goes live in order to e